In [34]:
# Import Dependencies

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from baseUnits import N, kN, tf, kgf, mm, cm, m, inches
from apeFunctions import rebarArea
from plotApeConfig import blueAPE, set_default_plot_params
set_default_plot_params()


# NonLinear Shell Equivalent Thickness Calculator

The purpose of this code is to create a calculator to define the nonlinear layer shell for concrete shear walls.

In [35]:
# Input

wall_thickness=400
cover=25
number_outPlane_int_points=5

number_smeared_layers=2
consider_bidirectional=True

steel_reinforcement=[[5,12,0],[5,16,90],[5,12,0],[5,16,90]]

# Material Tags
cover_material='UnconfinedConcrete'
center_material='ConfinedConcrete'
rebar_material='Steel'

# Define the width of the layers for plotting
layer_width = 1000

In [36]:
for reinforcement in steel_reinforcement:
    print(reinforcement)

[5, 12, 0]
[5, 16, 90]
[5, 12, 0]
[5, 16, 90]


In [37]:
# Calculations

first_layer_thickness=cover
last_layer_thickness=cover
mid_layer_thickness=(wall_thickness-2*cover)/(number_outPlane_int_points-2)

print(f'The mid layer thickness is: {round(mid_layer_thickness,2)}')

The mid layer thickness is: 116.67


In [38]:

# List to store thickness of each layer
layer_thickness = [first_layer_thickness] + [mid_layer_thickness] * (number_outPlane_int_points-2) + [last_layer_thickness]
layer_coord=np.cumsum(layer_thickness)

# Display the layer thicknesses
for i, thickness in enumerate(layer_thickness, 1):
    print(f"Layer {i} thickness: {thickness} mm")
    
print(layer_coord)

smeared_coord=np.linspace(cover,(wall_thickness-cover), number_smeared_layers)

print(f'The smeared_layers coordinates is: {smeared_coord}')

Layer 1 thickness: 25 mm
Layer 2 thickness: 116.66666666666667 mm
Layer 3 thickness: 116.66666666666667 mm
Layer 4 thickness: 116.66666666666667 mm
Layer 5 thickness: 25 mm
[ 25.         141.66666667 258.33333333 375.         400.        ]
The smeared_layers coordinates is: [ 25. 375.]


In [39]:
# Layer Shell Ensamble
matTag_vector=[]
thickness_vector=[]

if consider_bidirectional is True:
    number_smeared_layers_perLayer=2
else:
    number_smeared_layers_perLayer=1
    
total_numer_layers=number_outPlane_int_points+number_smeared_layers*number_smeared_layers_perLayer

print(f'The layer shell vector size is: {total_numer_layers}')

# Dataframe creation

data_dic={
    'Layer Name':[cover_material]+[center_material]*(number_outPlane_int_points-2)+[cover_material],
    'Layer Thickness':layer_thickness,
    'Begining Coordinate':[0]+list(layer_coord[:-1]),
    'End Coordinate':layer_coord
}

df=pd.DataFrame(data_dic)

The layer shell vector size is: 9


In [40]:
df

,Layer Name,Layer Thickness,Begining Coordinate,End Coordinate
0,UnconfinedConcrete,25.000000,0.000000,25.000000
1,ConfinedConcrete,116.666667,25.000000,141.666667
2,ConfinedConcrete,116.666667,141.666667,258.333333
3,ConfinedConcrete,116.666667,258.333333,375.000000
4,UnconfinedConcrete,25.000000,375.000000,400.000000


In [41]:


# Compute the cumulative thickness to get the y-coordinates
cumulative_thickness = np.cumsum([0] + layer_thickness)  # Starts from 0 (bottom of the first layer)

# Plot the layers
plt.figure(figsize=(8, 6))

# Loop through each layer to draw rectangles representing the layers
for i in range(len(layer_thicknesses)):
    # The bottom and top y-coordinates of the layer
    y_bottom = cumulative_thickness[i]
    y_top = cumulative_thickness[i+1]
    
    # Draw the rectangle (Layer) as a horizontal bar
    plt.fill_between([0, layer_width], y_bottom, y_top, color=blueAPE, edgecolor='k', alpha=0.6)
    plt.text(layer_width / 2, (y_bottom + y_top) / 2, f'Layer {i+1}', ha='center', va='center', fontsize=12, color='white')

# Set plot limits and labels
plt.xlim(0, layer_width)
plt.ylim(0, cumulative_thickness[-1])
plt.xlabel('Width (mm)')
plt.ylabel('Height (mm)')
plt.title('Layer Thickness Distribution in Coordinates')
plt.grid(True)
plt.show()


NameError: name 'layer_thicknesses' is not defined

<Figure size 800x600 with 0 Axes>